In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
import lightgbm as lgb

In [33]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [34]:
path = '/content/drive/My Drive/Colab Notebooks/PREDICT_FUTURE_SALES'
itemcat = pd.read_csv(path+'/item_categories.csv')
shops = pd.read_csv(path+'/shops.csv')
items = pd.read_csv(path+'/items.csv')
sale = pd.read_csv(path+'/sales_train.csv')
test = pd.read_csv(path+'/test.csv')

In [35]:
sale = reduce_mem_usage(sale)
test = reduce_mem_usage(test)

Memory usage of dataframe is 134.39 MB
Memory usage after optimization is: 33.65 MB
Decreased by 75.0%
Memory usage of dataframe is 4.90 MB
Memory usage after optimization is: 1.43 MB
Decreased by 70.8%


In [ ]:
sale.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [ ]:
sale.dtypes

date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

In [ ]:
sale.date = pd.to_datetime(sale.date,format = '%d.%m.%Y')

In [ ]:
itemcat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [ ]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [ ]:
sale.hist(figsize=(20,15),bins=50)

In [ ]:
sns.pairplot(sale)

In [ ]:
df = sale.groupby('date_block_num').sum()['item_cnt_day']
plt.figure(figsize = (20,4))
sns.barplot(x=df.index,y=df.values)
plt.title('Sales per month histogram')
plt.xticks(rotation= 45)
plt.xlabel('Price')
df = sale.groupby('date_block_num').sum()['item_cnt_day']
plt.figure(figsize = (20,4))
sns.lineplot(x=df.index,y=df.values,color='g')
plt.title('Sales per month')
plt.xlabel('Price')


In [ ]:
df = sale.groupby('shop_id').sum()['item_cnt_day']
plt.figure(figsize = (20,4))
sns.barplot(x=df.index,y=df.values)
plt.title('Sales per shop histogram')
plt.xticks(rotation= 45)
plt.xlabel('Price')
df = sale.groupby('shop_id').sum()['item_cnt_day']
plt.figure(figsize = (20,4))
sns.lineplot(x=df.index,y=df.values,color='g')
plt.title('Sales per shop')
plt.xlabel('Price')

# FEATURE GENERATION

In [36]:
itemcat['model'] = itemcat['item_category_name'].str.split(' - ').str[0]
itemcat['version'] = itemcat["item_category_name"].str.split(' - ').str[1]
itemcat.drop('item_category_name',axis=1,inplace=True)

In [37]:
shops['location'] = shops['shop_name'].str.split('"').str[1]
shops['shop_name'] = shops['shop_name'].str.split('"').str[0]

In [38]:
items.drop('item_name',axis=1,inplace=True)

In [39]:
sale = pd.merge(sale,items,how = 'left',on = 'item_id')
sale = pd.merge(sale,itemcat,how = 'left',on = 'item_category_id')
sale = pd.merge(sale,shops,how = 'left',on = 'shop_id')

In [40]:
test = pd.merge(test,items,how = 'left',on = 'item_id')
test = pd.merge(test,itemcat,how = 'left',on = 'item_category_id')
test = pd.merge(test,shops,how = 'left',on = 'shop_id')
test = pd.merge(test,sale[['item_id','item_price']],how='left',on='item_id')

In [41]:
sale = sale.loc[(sale.item_price<5000) & (sale.item_cnt_day<1000)]
sale['model'] = sale.model.astype(str)
sale['version'] = sale.version.astype(str)
sale['location'] = sale.location.astype(str)
sale['shop_name'] = sale.shop_name.astype(str)

In [42]:
trd = sale.drop(columns=['date','date_block_num'])
trd_cat = trd.drop(columns=['item_price','item_cnt_day'])
trd = trd.item_price
y_train = sale.item_cnt_day

In [43]:
del sale,itemcat,items,shops

In [44]:
ted = test.drop(columns=['ID'])

In [19]:
y_train.nunique()

195

In [ ]:
ted

In [ ]:
trd_cat

,shop_id,item_id,item_category_id,model,version,shop_name,location
0,59,22154,37,11,1,45,4
1,25,2552,58,13,29,22,6
2,25,2552,58,13,29,22,6
3,25,2554,58,13,29,22,6
4,25,2555,56,13,5,22,6
...,...,...,...,...,...,...,...
2935844,25,7409,55,13,4,22,6
2935845,25,7460,55,13,4,22,6
2935846,25,7459,55,13,4,22,6
2935847,25,7440,57,13,10,22,6


In [45]:
trd_cat.model = LabelEncoder().fit_transform(trd_cat.model) 
trd_cat.version = LabelEncoder().fit_transform(trd_cat.version) 
trd_cat.location = LabelEncoder().fit_transform(trd_cat.location) 
trd_cat.shop_name = LabelEncoder().fit_transform(trd_cat.shop_name) 
y_train = LabelEncoder().fit_transform(y_train)

In [46]:
trd = pd.concat([trd,trd_cat],axis=1)

In [47]:
trd

,item_price,shop_id,item_id,item_category_id,model,version,shop_name,location
0,999.000000,59,22154,37,11,1,45,4
1,899.000000,25,2552,58,13,29,22,6
2,899.000000,25,2552,58,13,29,22,6
3,1709.050049,25,2554,58,13,29,22,6
4,1099.000000,25,2555,56,13,5,22,6
...,...,...,...,...,...,...,...,...
2935844,299.000000,25,7409,55,13,4,22,6
2935845,299.000000,25,7460,55,13,4,22,6
2935846,349.000000,25,7459,55,13,4,22,6
2935847,299.000000,25,7440,57,13,10,22,6


In [48]:
del test

In [49]:
xtrain,xtest,ytrain,ytest = train_test_split(trd,y_train,test_size=0.2,random_state=441)

In [50]:
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

In [51]:
d_train = lgb.Dataset(xtrain, label= ytrain)

In [52]:
epochs = 100
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'multiclass'
params['metric'] = 'multi_logloss'
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
params['bagging_fraction']=0.8
params['lambda_l1']=0
params['num_class']=195
params['lambda_l1']=0

In [ ]:
model = lgb.train(params, d_train,num_boost_round = epochs)

In [ ]:
y_pred = model.predict(xtest)
rmse = np.sqrt(ytest,y_pred)
rmse

NameError: ignored